In [47]:
# scikit-learn                      0.23.2
# sklearn-crfsuite                  0.3.6
!pip install scikit-learn==0.23.2
!pip install sklearn-crfsuite==0.3.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
!pip install bnlp_toolkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
!git clone https://github.com/sagorbrur/bnlp/ --depth 1

fatal: destination path 'bnlp' already exists and is not an empty directory.


In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
# from bnlp import POS
# bn_pos = POS()
# model_path = "bnlp/model/bn_pos.pkl"
# text = ["আমি", 'ভাত', 'খাই'] # or you can pass ['আমি', 'ভাত', 'খাই', '।']
# for t in text:
#   res = bn_pos.tag(model_path, t)
#   print(res)


In [52]:
# dir(POS)

In [53]:
from bnlp.corpus import stopwords, punctuations, letters, digits

# print(stopwords)
# print(punctuations)
# print(letters)
# print(digits)

In [54]:
# from bnlp import NER
# bn_ner = NER()
# model_path = "bnlp/model/bn_ner.pkl"
# text = "সে ঢাকায় থাকে।" # or you can pass ['সে', 'ঢাকায়', 'থাকে', '।']
# result = bn_ner.tag(model_path, text)
# print(result)
# # [('সে', 'O'), ('ঢাকায়', 'S-LOC'), ('থাকে', 'O')]

In [55]:
# from bnlp import POS
# bn_pos = POS()
# model_path = "bnlp/model/bn_pos.pkl"
# text = ["ঢাকা ", 'হতে', 'দূরে'] # or you can pass ['আমি', 'ভাত', 'খাই', '।']
# for t in text:
#   res = bn_pos.tag(model_path, t)
#   print(res)


In [56]:

from bnlp import POS
bn_pos = POS()
model_path = "bnlp/model/bn_pos.pkl"

def isPPR(text): 
  if(bn_pos.tag(model_path, text) == "PPR"): 
    # print("Yes PPR")
    return True
  return False 

def isStopWord(text):
  global stopwords
  return text in stopwords

def isDigit(text):
  global digits
  return text in digits

def isPuncutuation(text): 
  global punctuations
  return text in punctuations


In [57]:
# from bnlp import POS
# bn_pos = POS()
# model_path = "bnlp/model/bn_pos.pkl"
# text = "ঢাকা হতে দূরে ।" # or you can pass ['আমি', 'ভাত', 'খাই', '।']
# res = bn_pos.tag(model_path, text)
# print(res)


In [58]:
# ! pip install -U 'scikit-learn<0.24'

In [59]:
# !pip install sklearn_crfsuite

In [60]:
!rm dev_generic_no_punc.csv
!rm train_generic_no_punc.csv

In [61]:
!cp "/content/drive/MyDrive/Colab Notebooks/NLP_Hackathon/dev_generic_no_punc.csv" dev_generic_no_punc.csv
!cp "/content/drive/MyDrive/Colab Notebooks/NLP_Hackathon/train_generic_no_punc.csv" train_generic_no_punc.csv

In [62]:
import pandas as pd

from sklearn_crfsuite import CRF
from sklearn_crfsuite import metrics

In [63]:
def extract_sentence_features(df):
    sentence_length = len(df)

    for i in range(sentence_length):
        token = df.iloc[i]['word']

        features = {
            'bias': 1.0,
            'token.lower()': token.lower(),
            # 'isPPR(token)': isPPR(token),
            'isStopWord(token)': isStopWord(token),
            'token.isupper()': token.isupper(),
            'token.istitle()': token.istitle(),
            # 'token.isdigit()': token.isdigit()
            'isDigit(token)': isDigit(token),
            'isPuncutuation(token)': isPuncutuation(token),
        }

        if i > 0:
            previous_token = df.iloc[i-1]['word']
            features.update({
                'previous_token.lower()': previous_token.lower(),
                # 'isPPR(previous_token)': isPPR(previous_token),
                'isStopWord(previous_token)': isStopWord(previous_token),
                'previous_token.isupper()': previous_token.isupper(),
                'previous_token.istitle()': previous_token.istitle(),
                # 'previous_token.isdigit()': previous_token.isdigit()
                'isDigit(previous_token)': isDigit(previous_token),
                'isPuncutuation(previous_token)': isPuncutuation(previous_token),
            })
        else:
            features['BOS'] = True

        if i < sentence_length - 1:
            posterior_token = df.iloc[i+1]['word']
            features.update({
                'posterior_token.lower()': posterior_token.lower(),
                # 'isPPR(posterior_token)': isPPR(posterior_token),
                'isStopWord(posterior_token)': isStopWord(posterior_token),
                'posterior_token.isupper()': posterior_token.isupper(),
                'posterior_token.istitle()': posterior_token.istitle(),
                # 'posterior_token.isdigit()': posterior_token.isdigit()
                'isDigit(posterior_token)': isDigit(posterior_token),
                'isPuncutuation(posterior_token)': isPuncutuation(posterior_token),
            })
        else:
            features['EOS'] = True

        yield features

In [64]:
from tqdm import tqdm
def prepare(df, include_y=False):
    X, y = [], []
    for _, group_df in tqdm(df.groupby(['sentence_id'])):
        X.append(list(extract_sentence_features(group_df)))
        if include_y:
            y.append(group_df['tag'])
    if include_y:
        return X, y
    return X

In [65]:
train_df = pd.read_csv('train_generic_no_punc.csv')
validation_df = pd.read_csv('dev_generic_no_punc.csv')


In [66]:
train_df.head()

,sentence_id,word,tag
0,0,তার,O
1,0,মৃত্যুর,O
2,0,দশ,O
3,0,দিন,O
4,0,পর,O


In [67]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191897 entries, 0 to 191896
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   sentence_id  191897 non-null  int64 
 1   word         191897 non-null  object
 2   tag          191897 non-null  object
dtypes: int64(1), object(2)
memory usage: 4.4+ MB


In [68]:
validation_df.head()

,sentence_id,word,tag
0,0,তিনি,O
1,0,যুবক,O
2,0,হিসেবে,O
3,0,শেফিল্ড,B-GRP
4,0,বুধবার,I-GRP


In [69]:
validation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10333 entries, 0 to 10332
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sentence_id  10333 non-null  int64 
 1   word         10333 non-null  object
 2   tag          10333 non-null  object
dtypes: int64(1), object(2)
memory usage: 242.3+ KB


In [70]:
train_df['word'].fillna('', inplace=True)
validation_df['word'].fillna('', inplace=True)


In [71]:
X_train, y_train = prepare(train_df, include_y=True)
X_validation, y_validation = prepare(validation_df, include_y=True)


100%|██████████| 800/800 [00:03<00:00, 207.36it/s]


In [72]:
len(X_train)

15300

In [73]:
len(X_validation)

800

In [74]:
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=500,
    all_possible_transitions=True,
    # verbose=True,
)

In [ ]:
crf.fit(X_train, y_train)
# try:
#     crf.fit(X_train, y_train)
# except AttributeError:
#     pass

In [ ]:
y_pred = crf.predict(X_validation)

In [ ]:
y_validation[0]

In [ ]:
y_pred[0]

In [ ]:
print(metrics.flat_classification_report(y_validation, y_pred, digits=4))

In [ ]:
# crf

In [ ]:
print("Done")

In [ ]:
import joblib

# save the model to a file
joblib.dump(crf, 'crf_model_no_punc_500_epoch.joblib')

# later, to load the model from the file
loaded_crf = joblib.load('crf_model_no_punc_500_epoch.joblib')


In [ ]:
# !cp crf_model.joblib "/content/drive/MyDrive/Colab Notebooks/NLP_Hackathon/"

In [ ]:
# print("RandomForest")

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(n_estimators = 100)


In [ ]:
# len(X_train)


In [ ]:
# X_train[0]

In [ ]:
# clf.fit(X_train, y_train)

In [ ]:
# y_pred = clf.predict(X_validation)

In [ ]:
# print(metrics.flat_classification_report(y_validation, y_pred, digits=4))